In [1]:
import os
import sys
from openai import OpenAI
from dotenv import load_dotenv

sys.path.append("../src")  # Add the directory containing `src`
from data_loader import DataLoader
from schema_generator import SchemaGenerator
from semantic_annotation import archetype_annotation
from join_discoverer import JoinDiscoverer

load_dotenv("/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/.env") # Insert the path to your .env file

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Or use your actual API key

/Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/matteocastagna/anaconda3/envs/archetype2/lib/python3.10/site-packages/torchvision/../../.

<h1>Eurostat<h1>

In [2]:
data_dir = "/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/data"
metadata_dir = "/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/data"
output_dir = "/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output"

# Initialization
data_loader = DataLoader(openai_client=openai_client)
schema_gen = SchemaGenerator(openai_client=openai_client)
join_dis = JoinDiscoverer(openai_client=openai_client)

In [3]:
datalake = data_loader.load_and_describe_datalake(
    data_directory=data_dir, # Directory with CSV or TSV
    metadata_directory=metadata_dir, # Optional directory with matadata in json format
    llm=True, # Produce a desciption with an LLM for each column
    sample_size=10, # How many values sample from a column
    output_directory=os.path.join(output_dir, "datalake") # Directory to save the output json
)

print(f"\nProcessed {len(datalake)} files")

[17:41:50] Starting data lake loading and description from /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/data


Processing data files:   0%|          | 0/4 [00:00<?, ?it/s]

[17:41:50] Found metadata for AACT_ALI02_data.csv: AACT_ALI02_metadata.json


Processing data files:  25%|██▌       | 1/4 [00:18<00:55, 18.43s/it]

[17:42:09] Found metadata for AACT_EAA06_data.csv: AACT_EAA06_metadata.json


...

[17:42:36] Found metadata for AACT_ALI01_data.csv: AACT_ALI01_metadata.json


Processing data files: 100%|██████████| 4/4 [00:57<00:00, 14.36s/it]


[17:42:48] Unifying column data...


Unifying descriptions with LLM: 100%|██████████| 12/12 [00:16<00:00,  1.36s/it]

[17:43:04] Column data unification complete.
[17:43:04] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/datalake
[17:43:04] Data lake JSON saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/datalake/data_lake.json
[17:43:04] Finished loading and describing 4 data files.

Processed 4 files


In [4]:
# Generate LinkML schema
schema = schema_gen.generate_linkml_schema(
    data_lake_list = datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    output_directory=os.path.join(output_dir, "schema") # Directory to save the output yaml
)

[17:43:04] Generating LinkML schema with LLM...
[17:43:18] Successfully generated LinkML schema.
[17:43:18] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/schema
[17:43:18] LinkML schema saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/schema/linkml_schema.yaml


In [5]:
# Prune LinkML schema
pruned_schema = schema_gen.prune_schema(
    data_lake_list = datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    yaml_schema = schema, # Either the dict produced by generate_linkml_schema or path to the yaml file produced
    output_directory=os.path.join(output_dir, "schema") # Directory to save the output yaml
)

[17:43:18] Starting schema pruning...
[17:43:18] Loaded YAML file with 12 class.attribute pairs and 14 descriptions.


Classifying datasets for pruning: 100%|██████████| 4/4 [00:18<00:00,  4.52s/it]

[17:43:36] Total unique relevant classes identified: 1
[17:43:36] Relevant classes: ['Document']
[17:43:36] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/schema
[17:43:36] Pruned LinkML schema saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/schema/linkml_schema_pruned.yaml


In [6]:
# Semnaitc column annotaiton with ArcheType
datalake_annotated = archetype_annotation(
    data_lake_list=datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    yaml_schema=pruned_schema,  # Either the dict produced by generate_linkml_schema/prune_schema or path to the yaml file produced
    sample_size=10, # How many samples of the column pass to Archetype for the annotation
    output_directory=os.path.join(output_dir, "datalake") # Directory to save the output json
)

[17:43:36] Starting semantic annotation with Archetype...
[17:43:36] Loaded YAML file with 12 class.attribute pairs and 14 descriptions.
[17:43:36] Loaded YAML schema from for Archetype.


Annotating files:   0%|          | 0/4 [00:00<?, ?it/s]

[17:43:36] Processing file: AACT_ALI02_data.csv


Annotating files:  25%|██▌       | 1/4 [00:05<00:16,  5.62s/it]

[17:43:41] Processing file: AACT_EAA06_data.csv


...

[17:43:48] Processing file: ACF_D_EQ1_data.csv


Annotating files:  75%|███████▌  | 3/4 [00:21<00:07,  7.69s/it]

[17:43:58] Processing file: AACT_ALI01_data.csv


Annotating files: 100%|██████████| 4/4 [00:28<00:00,  7.01s/it]

[17:44:04] Finished semantic annotation for 4 files.
[17:44:04] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/datalake
[17:44:04] Annotated data lake JSON saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/datalake/data_lake_annotated.json


In [7]:
# Generate embeddings for each column
embeddings = join_dis.generate_embeddings(
    data_lake_list=datalake_annotated, # Either the list of dict produced by load_and_describe_datalake/archetype_annotation or the path to the json file produced
    output_directory=os.path.join(output_dir, "embeddings") # Directory to save the output json
)

[17:44:04] Generating Embeddings: 100%|██████████| 37/37 [00:17<00:00,  2.09it/s]

[17:44:22] Finished generating embeddings for 37 columns.
[17:44:22] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/embeddings
[17:44:22] Embeddings saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/embeddings/embeddings.json


In [8]:
# Generate Neo4j graph
_, _, _ = join_dis.compute_distances_and_export_neo4j(
    embeddings=embeddings, # Either the list of dict produced by generate_embeddings or the path to the json file produced
    cosine_sim_threshold = 0.5,
    anns_threshold = 0.2,
    output_directory=os.path.join(output_dir, "neo4j") # Directory to save the output CSVs
)

[17:44:22] Calculating cosine similarity and euclidean distances...
[17:44:22] Calculating ANNS distances...
[17:44:22] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/neo4j
[17:44:22] Distances saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/neo4j/distances.csv
[17:44:22] Generating Neo4j nodes CSV...
[17:44:22] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/neo4j
[17:44:22] Neo4j nodes saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/neo4j/nodes.csv
[17:44:22] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/eurostat/output/neo4j
[17:44:22] Neo4j edges saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/dem

<h1>PKT<h1>

In [9]:
data_dir = "/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/data"
output_dir = "/Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output"

# Initialization
data_loader = DataLoader(openai_client=openai_client)
schema_gen = SchemaGenerator(openai_client=openai_client)
join_dis = JoinDiscoverer(openai_client=openai_client)

In [10]:
datalake = data_loader.load_and_describe_datalake(
    data_directory=data_dir, # Directory with CSV or TSV
    llm=True, # Produce a desciption with an LLM for each column
    sample_size=10, # How many values sample from a column
    output_directory=os.path.join(output_dir, "datalake") # Directory to save the output json
)

print(f"\nProcessed {len(datalake)} files")

[17:44:22] Starting data lake loading and description from /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/data


Processing data files: 100%|██████████| 7/7 [00:58<00:00,  8.32s/it]


[17:45:20] Unifying column data...


Unifying descriptions with LLM: 100%|██████████| 16/16 [00:01<00:00,  9.64it/s]

[17:45:22] Column data unification complete.
[17:45:22] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/datalake
[17:45:22] Data lake JSON saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/datalake/data_lake.json
[17:45:22] Finished loading and describing 7 data files.

Processed 7 files


In [11]:
# Generate LinkML schema
schema = schema_gen.generate_linkml_schema(
    data_lake_list = datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    output_directory=os.path.join(output_dir, "schema") # Directory to save the output yaml
)

[17:45:22] Generating LinkML schema with LLM...
[17:45:55] Successfully generated LinkML schema.
[17:45:55] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/schema
[17:45:55] LinkML schema saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/schema/linkml_schema.yaml


In [12]:
# Prune LinkML schema
pruned_schema = schema_gen.prune_schema(
    data_lake_list = datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    yaml_schema = schema, # Either the dict produced by generate_linkml_schema or path to the yaml file produced
    output_directory=os.path.join(output_dir, "schema") # Directory to save the output yaml
)

[17:45:55] Starting schema pruning...
[17:45:55] Loaded YAML file with 16 class.attribute pairs and 24 descriptions.


Classifying datasets for pruning: 100%|██████████| 7/7 [00:42<00:00,  6.03s/it]

[17:46:37] Total unique relevant classes identified: 8
[17:46:37] Relevant classes: ['Chemical', 'NamedEntity', 'Cofactor', 'Gene', 'Protein', 'Catalyst', 'Phenotype', 'Document']
[17:46:37] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/schema
[17:46:37] Pruned LinkML schema saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/schema/linkml_schema_pruned.yaml


In [13]:
# Semnaitc column annotaiton with ArcheType
datalake_annotated = archetype_annotation(
    data_lake_list=datalake, # Either the list of dict produced by load_and_describe_datalake or the path to the json file produced
    yaml_schema=pruned_schema,  # Either the dict produced by generate_linkml_schema/prune_schema or path to the yaml file produced
    sample_size=10, # How many samples of the column pass to Archetype for the annotation
    output_directory=os.path.join(output_dir, "datalake") # Directory to save the output json
)

[17:46:37] Starting semantic annotation with Archetype...
[17:46:37] Loaded YAML file with 16 class.attribute pairs and 24 descriptions.
[17:46:37] Loaded YAML schema from for Archetype.


Annotating files:   0%|          | 0/7 [00:00<?, ?it/s]

[17:46:37] Processing file: ENSEMBL_GENE_ENTREZ_GENE_MAP.tsv


Annotating files:  14%|█▍        | 1/7 [00:05<00:30,  5.16s/it]

[17:46:43] Processing file: UNIPROT_PROTEIN_COFACTOR.tsv


...

[17:46:50] Processing file: STRING_PRO_ONTOLOGY_MAP.tsv


Annotating files:  86%|████████▌ | 6/7 [00:14<00:02,  2.00s/it]

[17:46:51] Processing file: PHENOTYPE_HPO_MAP.tsv


Annotating files: 100%|██████████| 7/7 [00:15<00:00,  2.26s/it]

[17:46:53] Finished semantic annotation for 7 files.
[17:46:53] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/datalake
[17:46:53] Annotated data lake JSON saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/datalake/data_lake_annotated.json


In [14]:
# Generate embeddings for each column
embeddings = join_dis.generate_embeddings(
    data_lake_list=datalake_annotated, # Either the list of dict produced by load_and_describe_datalake/archetype_annotation or the path to the json file produced
    output_directory=os.path.join(output_dir, "embeddings") # Directory to save the output json
)

[17:46:53] Generating Embeddings: 100%|██████████| 19/19 [00:08<00:00,  2.22it/s]

[17:47:02] Finished generating embeddings for 19 columns.
[17:47:02] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/embeddings
[17:47:02] Embeddings saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/embeddings/embeddings.json


In [15]:
# Generate Neo4j graph
_, _, _ = join_dis.compute_distances_and_export_neo4j(
    embeddings=embeddings, # Either the list of dict produced by generate_embeddings or the path to the json file produced
    cosine_sim_threshold = 0.5,
    anns_threshold = 0.2,
    output_directory=os.path.join(output_dir, "neo4j") # Directory to save the output CSVs
)

[17:47:02] Calculating cosine similarity and euclidean distances...
[17:47:02] Calculating ANNS distances...
[17:47:02] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j
[17:47:02] Distances saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j/distances.csv
[17:47:02] Generating Neo4j nodes CSV...
[17:47:02] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j
[17:47:02] Neo4j nodes saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j/nodes.csv
[17:47:02] Ensured directory exists: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j
[17:47:02] Neo4j edges saved to: /Users/matteocastagna/Documents/Università/Assegno di ricerca 2024:2025/SemLink/demo/pkt/output/neo4j/edges.